# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,klyuchi,52.2667,79.1667,19.02,75,96,4.90,RU,1689976354
1,1,grytviken,-54.2811,-36.5092,3.18,97,100,6.62,GS,1689976355
2,2,codrington,-38.2667,141.9667,9.46,87,100,5.15,AU,1689976081
3,3,seme,15.1942,-12.9448,32.27,51,95,5.27,SN,1689976355
4,4,pointe aux piments,-20.0628,57.5131,22.31,77,8,5.75,MU,1689976355


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City")


# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df[(city_data_df["Max Temp"] <=28) & (city_data_df["Humidity"]<50)]

# Drop any rows with null values
city_data_df = city_data_df.dropna(how="any")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,yellowknife,62.4560,-114.3525,27.29,34,100,6.17,CA,1689976646
33,33,bilibino,68.0546,166.4372,22.76,43,39,0.78,RU,1689976728
35,35,whitehorse,60.7161,-135.0538,26.01,39,75,2.06,CA,1689976499
80,80,farsan,32.2569,50.5610,22.02,15,6,2.20,IR,1689976371
84,84,ghormach,35.7306,63.7826,25.63,21,0,2.37,AF,1689976372


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df =  city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,yellowknife,CA,62.4560,-114.3525,34,
33,bilibino,RU,68.0546,166.4372,43,
35,whitehorse,CA,60.7161,-135.0538,39,
80,farsan,IR,32.2569,50.5610,15,
84,ghormach,AF,35.7306,63.7826,21,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":20,
    "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yellowknife - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
farsan - nearest hotel: No hotel found
ghormach - nearest hotel: No hotel found
kununurra - nearest hotel: Kununurra Country Club Resort
carnarvon - nearest hotel: No hotel found
hastings - nearest hotel: White Rock Hotel
esquina - nearest hotel: La Casona Pesca
esfarayen - nearest hotel: هتل ساریگل
baiyin - nearest hotel: 红鹭宾馆
dalandzadgad - nearest hotel: Хан Уул буудал
badger - nearest hotel: Hotel North Pole
senillosa - nearest hotel: No hotel found
balqash - nearest hotel: Престиж
aripuana - nearest hotel: No hotel found
la ronge - nearest hotel: La Ronge Hotel and Suites
kapuskasing - nearest hotel: Travelodge
la sarre - nearest hotel: No hotel found
camacupa - nearest hotel: No hotel found
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
shahrak - nearest hotel: No hotel found
flin flon - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
12,yellowknife,CA,62.4560,-114.3525,34,No hotel found
33,bilibino,RU,68.0546,166.4372,43,No hotel found
35,whitehorse,CA,60.7161,-135.0538,39,Town & Mountain Hotel
80,farsan,IR,32.2569,50.5610,15,No hotel found
84,ghormach,AF,35.7306,63.7826,21,No hotel found
92,kununurra,AU,-15.7667,128.7333,43,Kununurra Country Club Resort
145,carnarvon,AU,-24.8667,113.6333,36,No hotel found
151,hastings,GB,50.8552,0.5729,40,White Rock Hotel
152,esquina,AR,-30.0144,-59.5272,45,La Casona Pesca
203,esfarayen,IR,37.0765,57.5101,24,هتل ساریگل


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .75,
    color = "City",
    hover_cols = ["Hotel Name", "Country"])


# Display the map
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)